# PyNNcml Setup and Integration Tests (Cross-Platform)

This notebook verifies PyNNcml setup for the OpenMesh project on **Windows, Mac, and Linux**.

## What This Notebook Does

1. **Automatically detects project root** (no hardcoded paths)
2. **Checks PyNNcml directory** structure and files
3. **Verifies dependencies** are installed
4. **Confirms editable install** (works correctly on Windows)
5. **Tests PyNNcml imports** and basic functionality

## Prerequisites

- Python environment activated (conda/venv)
- PyNNcml directory in project root
- Git repository cloned

## Quick Start

Run all cells in order. If any test fails, follow the instructions provided.

## Setup: Detect Project Root and Paths

This cell automatically finds the project root by looking for the PyNNcml directory.

In [ ]:
# ============================================================================
# SETUP: Dynamic Project Root Detection (Cross-Platform)
# ============================================================================
import sys
import os
from pathlib import Path
from datetime import datetime
import platform

print("=" * 70)
print("SETUP: Detecting Project Structure")
print("=" * 70)

# Detect operating system
os_name = platform.system()
print(f"\nOperating System: {os_name}")
print(f"Python Version: {platform.python_version()}")
print(f"Current Directory: {Path.cwd()}")

# Find project root by walking up until we find PyNNcml directory
current_dir = Path.cwd()
project_root = current_dir

# Walk up the directory tree
while project_root != project_root.parent:
    if (project_root / 'PyNNcml').exists():
        # Found it!
        break
    project_root = project_root.parent

# Check if we found PyNNcml
if not (project_root / 'PyNNcml').exists():
    print("\n⚠ Could not find PyNNcml directory")
    print(f"  Searched from: {current_dir}")
    print(f"  Up to: {project_root}")
    project_root = current_dir
else:
    print(f"\n✓ Project root detected: {project_root}")

# Set up paths
pynncml_dir = project_root / 'PyNNcml'
print(f"✓ PyNNcml directory: {pynncml_dir}")
print(f"✓ PyNNcml exists: {pynncml_dir.exists()}")

print("\n" + "=" * 70)

## Test 1: Check PyNNcml Directory Structure

In [ ]:
# ============================================================================
# TEST 1: Check PyNNcml Directory Structure
# ============================================================================
print("=" * 70)
print("TEST 1: PyNNcml Directory Structure")
print("=" * 70)

# Check critical files
setup_py = pynncml_dir / 'setup.py'
pyproject_toml = pynncml_dir / 'pyproject.toml'
pynncml_package = pynncml_dir / 'pynncml'
pynncml_init = pynncml_package / '__init__.py'

checks = {
    'PyNNcml directory': pynncml_dir.exists(),
    'setup.py': setup_py.exists(),
    'pyproject.toml': pyproject_toml.exists(),
    'pynncml package': pynncml_package.exists() and pynncml_package.is_dir(),
    'pynncml/__init__.py': pynncml_init.exists()
}

print("\nDirectory structure check:")
all_good = True
for name, exists in checks.items():
    status = "✓" if exists else "✗"
    print(f"  {status} {name}")
    if not exists and name != 'pyproject.toml':  # pyproject.toml is optional
        all_good = False

# Determine if PyNNcml is ready
pynncml_ready = checks['PyNNcml directory'] and checks['pynncml package'] and (checks['setup.py'] or checks['pyproject.toml'])

print("\n" + "=" * 70)
if pynncml_ready:
    print("Result: ✓ PyNNcml directory structure is valid")
else:
    print("Result: ✗ PyNNcml directory structure is incomplete")
    print("\nTo fix:")
    print(f"  1. Clone PyNNcml repository:")
    print(f"     cd {project_root}")
    print(f"     git clone git@github.com:drorjac/PyNNcml.git PyNNcml")
print("=" * 70)

## Test 2: Check Dependencies

In [ ]:
# ============================================================================
# TEST 2: Check Dependencies
# ============================================================================
print("=" * 70)
print("TEST 2: Dependency Check")
print("=" * 70)

if pynncml_ready:
    # Define required packages
    required_packages = {
        'numpy': 'numpy',
        'torch': 'torch',
        'pandas': 'pandas',
        'xarray': 'xarray',
        'matplotlib': 'matplotlib',
        'scipy': 'scipy',
        'netcdf4': 'netCDF4'
    }
    
    missing_packages = []
    installed_packages = []
    
    print("\nChecking required packages:")
    for package_name, import_name in required_packages.items():
        try:
            __import__(import_name)
            installed_packages.append(package_name)
            print(f"  ✓ {package_name}")
        except ImportError:
            missing_packages.append(package_name)
            print(f"  ✗ {package_name} - NOT INSTALLED")
    
    # Summary
    print(f"\n{'='*70}")
    print(f"Summary: {len(installed_packages)}/{len(required_packages)} packages installed")
    
    if missing_packages:
        print(f"\n⚠ Missing: {', '.join(missing_packages)}")
        print("\nTo install:")
        print(f"  pip install {' '.join(missing_packages)}")
        print("\nOr install from requirements:")
        print(f"  pip install -r {project_root / 'requirements.txt'}")
        deps_ok = False
    else:
        print("\n✓ All required dependencies are installed!")
        deps_ok = True
    
    # Check requirements files
    print(f"\n{'='*70}")
    print("Requirements files:")
    project_req = project_root / 'requirements.txt'
    pynncml_req = pynncml_dir / 'requirements.txt'
    
    if project_req.exists():
        print(f"  ✓ Project: {project_req}")
    else:
        print(f"  ✗ Project requirements.txt not found")
    
    if pynncml_req.exists():
        print(f"  ✓ PyNNcml: {pynncml_req}")
    else:
        print(f"  ✗ PyNNcml requirements.txt not found")
        
else:
    print("\n⚠ Skipping dependency check (PyNNcml directory not ready)")
    deps_ok = False

print("=" * 70)

## Test 3: Import PyNNcml and Check Install Type

In [ ]:
# ============================================================================
# TEST 3: Import PyNNcml (Cross-Platform)
# ============================================================================
print("=" * 70)
print("TEST 3: PyNNcml Import Test")
print("=" * 70)

try:
    import pynncml as pnc
    print("\n✓ Successfully imported pynncml")
    print(f"  Module location: {pnc.__file__}")
    
    # Get version
    version = getattr(pnc, '__version__', 'unknown')
    print(f"  Version: {version}")
    
    # Check if editable install (CROSS-PLATFORM FIX)
    # Use case-insensitive comparison for Windows compatibility
    pnc_path_lower = str(pnc.__file__).lower()
    project_root_lower = str(project_root).lower()
    
    if project_root_lower in pnc_path_lower:
        print("\n✓ EDITABLE INSTALL CONFIRMED")
        print("  Source location: Project directory")
        print("  Changes to PyNNcml source will be immediately available")
        is_editable = True
    else:
        print("\n⚠ NOT EDITABLE (installed in site-packages)")
        print("  Source location: Python site-packages")
        print("  Changes to PyNNcml source will NOT be available")
        print("\n  To enable editable install:")
        print(f"    cd {pynncml_dir}")
        print(f"    pip install -e .")
        is_editable = False
    
    import_success = True
    
except ImportError as e:
    print(f"\n✗ Failed to import pynncml")
    print(f"  Error: {e}")
    print("\nTo install PyNNcml:")
    print(f"  cd {pynncml_dir}")
    print(f"  pip install -e .")
    import_success = False
    is_editable = False
    pnc = None

print("\n" + "=" * 70)

## Test 4: Verify Source File Access (Cross-Platform)

In [ ]:
# ============================================================================
# TEST 4: Verify Source File Access (UTF-8, Cross-Platform)
# ============================================================================
print("=" * 70)
print("TEST 4: Source File Access Verification")
print("=" * 70)

if pynncml_ready:
    init_file = pynncml_dir / 'pynncml' / '__init__.py'
    
    if init_file.exists():
        print(f"\n✓ Found source file: {init_file}")
        
        try:
            # Read with UTF-8 encoding (works on all platforms)
            with open(init_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            print(f"  File size: {init_file.stat().st_size} bytes")
            print(f"  Last modified: {datetime.fromtimestamp(init_file.stat().st_mtime)}")
            print(f"  Lines of code: {len(content.splitlines())}")
            
            print("\n✓ Can read and access PyNNcml source files")
            
        except UnicodeDecodeError as e:
            print(f"\n⚠ Encoding error: {e}")
            print("  Trying fallback with error handling...")
            with open(init_file, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
            print("  ✓ Read successful with fallback")
            
        except Exception as e:
            print(f"\n✗ Error reading file: {e}")
    else:
        print(f"\n✗ Source file not found: {init_file}")
else:
    print("\n⚠ Skipping test (PyNNcml directory not ready)")

print("\n" + "=" * 70)

## Test 5: Check PyNNcml Modules

In [ ]:
# ============================================================================
# TEST 5: Check PyNNcml Module Structure
# ============================================================================
print("=" * 70)
print("TEST 5: PyNNcml Module Structure")
print("=" * 70)

if import_success and pnc is not None:
    print("\nChecking key PyNNcml modules:")
    
    # Key modules to check
    modules_to_check = [
        ('utils', 'Utility functions'),
        ('datasets', 'Dataset handling'),
        ('neural_networks', 'Neural network models'),
        ('metrics', 'Evaluation metrics')
    ]
    
    available_modules = []
    for module_name, description in modules_to_check:
        if hasattr(pnc, module_name):
            available_modules.append(module_name)
            print(f"  ✓ {module_name} - {description}")
        else:
            print(f"  ✗ {module_name} - Not found")
    
    print(f"\n{'='*70}")
    print(f"Available: {len(available_modules)}/{len(modules_to_check)} core modules")
    
    if len(available_modules) >= 3:
        print("\n✓ PyNNcml modules are accessible")
    else:
        print("\n⚠ Some PyNNcml modules are missing")
        print("  This may indicate an incomplete installation")
else:
    print("\n⚠ Skipping test (PyNNcml not imported)")

print("\n" + "=" * 70)

## Test 6: Verify Editable Install with pip show

In [ ]:
# ============================================================================
# TEST 6: Verify Installation with pip show
# ============================================================================
print("=" * 70)
print("TEST 6: Pip Installation Check")
print("=" * 70)

import subprocess

try:
    result = subprocess.run(
        ['pip', 'show', 'pynncml'],
        capture_output=True,
        text=True,
        check=True
    )
    
    print("\nPip package information:")
    print("-" * 70)
    
    # Parse and display key information
    for line in result.stdout.split('\n'):
        if any(key in line for key in ['Name:', 'Version:', 'Location:', 'Editable project']):
            print(f"  {line}")
    
    print("-" * 70)
    
    # Check if editable
    if 'Editable project location:' in result.stdout:
        print("\n✓ Confirmed: Editable install detected by pip")
    else:
        print("\n⚠ Not an editable install (pip check)")
        
except subprocess.CalledProcessError:
    print("\n⚠ PyNNcml not found by pip")
    print("  Package may not be installed")
except FileNotFoundError:
    print("\n⚠ pip command not found")

print("\n" + "=" * 70)

## Summary and Recommendations

In [ ]:
# ============================================================================
# SUMMARY: Overall Test Results
# ============================================================================
print("=" * 70)
print("TEST SUMMARY")
print("=" * 70)

# Collect all test results
tests_status = []

# Check variables exist
if 'pynncml_ready' not in globals():
    pynncml_ready = False
if 'deps_ok' not in globals():
    deps_ok = False
if 'import_success' not in globals():
    import_success = False
if 'is_editable' not in globals():
    is_editable = False

# Display results
print("\nTest Results:")
print(f"  {'✓' if pynncml_ready else '✗'} PyNNcml directory structure")
print(f"  {'✓' if deps_ok else '⚠'} Dependencies installed")
print(f"  {'✓' if import_success else '✗'} PyNNcml import")
print(f"  {'✓' if is_editable else '⚠'} Editable install")

print("\n" + "=" * 70)

# Overall status
if import_success and is_editable:
    print("\n🎉 SUCCESS: PyNNcml is properly set up!")
    print("\nYou can now:")
    print("  • Import PyNNcml in your notebooks")
    print("  • Modify PyNNcml source code")
    print("  • Changes will be immediately available (just restart kernel)")
    print("\nNext steps:")
    print("  • Start using PyNNcml for your analysis")
    print("  • See project documentation for examples")
    
elif import_success and not is_editable:
    print("\n⚠ PARTIAL SUCCESS: PyNNcml works but not in editable mode")
    print("\nTo enable editable mode (recommended for development):")
    print(f"  1. Open terminal")
    print(f"  2. cd {pynncml_dir}")
    print(f"  3. pip uninstall pynncml")
    print(f"  4. pip install -e .")
    print(f"  5. Restart this notebook kernel")
    
elif pynncml_ready and not import_success:
    print("\n⚠ PyNNcml directory found but import failed")
    print("\nTo fix:")
    print(f"  1. cd {pynncml_dir}")
    if not deps_ok:
        print(f"  2. pip install -r requirements.txt")
        print(f"  3. pip install -e .")
    else:
        print(f"  2. pip install -e .")
    print(f"  4. Restart this notebook kernel")
    
else:
    print("\n✗ Setup incomplete")
    print("\nTo set up PyNNcml:")
    print(f"  1. cd {project_root}")
    print(f"  2. git clone git@github.com:drorjac/PyNNcml.git PyNNcml")
    print(f"  3. cd PyNNcml")
    print(f"  4. pip install -r requirements.txt")
    print(f"  5. pip install -e .")
    print(f"  6. Re-run this notebook")

print("\n" + "=" * 70)

# System info for debugging
print("\nSystem Information (for debugging):")
print(f"  OS: {os_name}")
print(f"  Python: {platform.python_version()}")
print(f"  Project root: {project_root}")
if import_success:
    print(f"  PyNNcml location: {pnc.__file__}")

print("\n" + "=" * 70)